In [2]:
import os.path
import pandas as pd
import clb
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import typhon.plots
import seaborn as sns

plt.style.use('typhon')

In [3]:
BASE_DIR = '/home/tfinn/Data/Ceilometer'
ceilo_first = clb.csv.read(os.path.join(BASE_DIR, 'LEX', 'CL_M10_first.txt'))
ceilo_second = clb.csv.read(os.path.join(BASE_DIR, 'LEX', 'CL_M10_first.txt'))
ceilo_first['DATETIME'] = np.array([dt.datetime.strptime('-'.join([D, ceilo_first['TIME'][k]]), '%d.%m.%Y-%H:%M') for k, D in enumerate(ceilo_first['DATE'])])
ceilo_second['DATETIME'] = np.array([dt.datetime.strptime('-'.join([D, ceilo_second['TIME'][k]]), '%d.%m.%Y-%H:%M') for k, D in enumerate(ceilo_second['DATE'])])

In [14]:
ceilo = pd.DataFrame(ceilo_first).set_index('DATETIME')
cam = pd.read_json('/home/tfinn/Projects/pyclamster/data/heights_201609010900_4h_300.json')
ceilo.index = ceilo.index-dt.timedelta(hours=2)
cam.index = cam.index-dt.timedelta(hours=1)
cam[cam>8000] = np.NaN
cam_min = cam.mean(axis=1)

In [15]:
ceilo_plt = ceilo.ix['2016/09/01 06:30':'2016/09/01 14:30', ['CL_SCHA_M10', 'CL_SCHB_M10']]
ceilo_cam = pd.DataFrame()
ceilo_cam = ceilo_plt
ceilo_cam.ix[:, 'cloud camera'] = cam_min.resample('10min').mean()
ceilo_cam.dropna().corr()

,CL_SCHA_M10,CL_SCHB_M10,cloud camera
CL_SCHA_M10,1.000000,0.493732,0.417173
CL_SCHB_M10,0.493732,1.000000,-0.057982
cloud camera,0.417173,-0.057982,1.000000


In [ ]:
fig, ax = plt.subplots()
fig.suptitle('Ceilometer and cloud camera')
c = ceilo_cam['CL_SCHA_M10'].plot(ax=ax, label='ceilo 1')
c = ceilo_cam['CL_SCHB_M10'].plot(ax=ax, label='ceilo 2')
s = ax.scatter(ceilo_cam.index, ceilo_cam['cloud camera'], c='r')
plt.legend()
ax.set_ylim(0,9000)
ax.set_xlabel('Time 01/09/2016 in UTC+1')
ax.set_ylabel('Height [m]')
plt.tight_layout()
plt.savefig('/home/tfinn/Desktop/ceilo_without.png')
fig, ax = plt.subplots()
fig.suptitle('Scatter ceilo/ cloud camera')
g = sns.jointplot("CL_SCHA_M10", "cloud camera", data=ceilo_cam, kind='reg',
                  xlim=(0, 8000), ylim=(0, 8000), color="r", size=8)
plt.show()

/home/tfinn/anaconda3/lib/python3.5/site-packages/statsmodels/nonparametric/kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


In [ ]:
plt.savefig('/home/tfinn/Desktop/scatter_without.png')
plt.close('all')